In [1]:
import os, sys
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy.stats as stats
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime as dt
import astropy as ap
from netCDF4 import Dataset, MFDataset
import h5py
import xarray as xr
from mpl_toolkits.mplot3d import axes3d
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.colors as mcolors
import cftime
import matplotlib.lines as mlines
from sklearn.decomposition import PCA
from sklearn import preprocessing, cluster
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cdist
import csv
from itertools import product
from sklearn.ensemble import RandomForestRegressor
import matplotlib.patches as patches
import statsmodels.api as sm
import pymannkendall as mk

In [27]:
grades_df.head()

,Student,ID,SIS User ID,SIS Login ID,Section,Midterm (1609301),Homework-1 (1591450),Homework-2 (1602448),Homework-3 (1609751),HW4 (1613490),...,Current Points,Final Points,Current Score,Unposted Current Score,Final Score,Unposted Final Score,Current Grade,Unposted Current Grade,Final Grade,Unposted Final Grade
0,"AGUILAR, ANDREA",87618.0,505962887.0,andrea1212@ucla.edu,24S-A&O SCI-1-DIS-1C,26.0,0.0,NaN,31.0,NaN,...,830.12,830.12,83.26,83.26,77.80,77.80,B,B,C+,C+
1,"ALLEN, MAYA",120268.0,6214844.0,mayaallen23@ucla.edu,24S-A&O SCI-1-DIS-1A,30.0,23.0,27.5,32.0,39.0,...,906.70,906.70,84.98,84.98,84.98,84.98,B,B,B,B
2,"ANDERSON, LAUREL",117972.0,206197037.0,landerson2597@ucla.edu,24S-A&O SCI-1-DIS-1E,35.0,26.0,29.0,32.0,38.5,...,1022.02,1022.02,95.78,95.78,95.78,95.78,A,A,A,A
3,"ARAGON, GABY",118283.0,306143845.0,gearagon23@ucla.edu,24S-A&O SCI-1-DIS-1C,35.0,27.0,27.0,31.0,35.0,...,996.00,996.00,93.35,93.35,93.35,93.35,A,A,A,A
4,"AYAAY, VICTOR",122225.0,306145496.0,vayaay@ucla.edu,24S-A&O SCI-1-DIS-1G,32.5,25.5,24.0,32.0,34.0,...,907.64,907.64,93.86,93.86,85.06,85.06,A,A,B,B


In [107]:
grades_df = pd.read_csv('2024-06-18T1524_Grades-24S-A&O_SCI-1-LEC-1.csv')

#dropping first row which is all nans
grades_df = grades_df.iloc[1:].reset_index(drop=True)

student_names = grades_df.iloc[:, 0]

id_ = grades_df.iloc[:, 1]

SISUserID = grades_df.iloc[:, 2]

SISLoginID = grades_df.iloc[:, 3]

Section = grades_df.iloc[:, 4]

# Homework
HW = grades_df.iloc[:, 6:10].to_numpy()

HWt = grades_df.iloc[0, 6:10].to_numpy()

HW[np.isnan(HW)] = 0

# Quiz
Q = grades_df.iloc[:, 10:18].to_numpy()

Q = np.array(Q, dtype=float)

Qt = grades_df.iloc[0, 10:18].to_numpy()

Qt = Qt.astype(float)

Q[np.isnan(Q)] = 0

# For CHRISTOPHER GARCIA give him credit for the first four quizzes
Q[34, [0, 1, 3]] = 100

# Midterm
MT = grades_df.iloc[:, 5].to_numpy()

MTt = grades_df.iloc[0, 5]

MT[np.isnan(MT)] = 0

# Final
F = grades_df.iloc[:, 18].to_numpy()

Ft = grades_df.iloc[0, 18]

F[np.isnan(F)] = 0

# Extra-credit: participation
attendance_df = pd.read_csv('attendance.csv')

attendance_df = attendance_df.iloc[1:].reset_index(drop=True)

grades_df = grades_df.iloc[1:].reset_index(drop=True)

EC1 = attendance_df.iloc[:, 1].to_numpy()  # lecture 17

EC1[np.isnan(EC1)] = 0

EC2 = attendance_df.iloc[:, 2].to_numpy()  # lecture 13

EC2[np.isnan(EC2)] = 0

EC3 = attendance_df.iloc[:, 3].to_numpy()  # total

EC3[np.isnan(EC3)] = 0

edges = [0, 60, 63, 67, 70, 73, 77, 80, 83, 87, 90, 93, 97, 120]

letter = ['F', 'D-', 'D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+']

In [109]:
# Grade in BruinLearn
# Final_grade_BL = ((np.nansum(HW, axis=1) + np.nansum(Q, axis=1) + np.nansum(MT) + np.nansum(F)) / 
#                   (np.nansum(HWt) + np.nansum(Qt) + np.nansum(MTt) + np.nansum(Ft))) * 100

Final_grade_BL = ((np.nansum(HW, axis=1) + np.nansum(Q, axis=1) + MT + F)) / (np.nansum(HWt) + np.nansum(Qt) + np.nansum(MTt) + np.nansum(Ft)) * 100

# letter_BL = np.array(['']*len(Final_grade_BL))
letter_BL = np.full(len(Final_grade_BL), '', dtype=object)

for j in range(len(Final_grade_BL)):
    
    for i in range(len(letter)):
        
        if edges[i] <= Final_grade_BL[j] < edges[i+1]:
            
            letter_BL[j] = letter[i]
            
            break
            

# Real grade with removal
BLHW = np.zeros(len(F))

BLQ = np.zeros(len(F))

BLF = np.zeros(len(F))

BLMT = np.zeros(len(F))

BEC = np.zeros(len(F))

for i in range(len(F)):
    # Calculate HW, drop 1
    auxHW = np.sort(HW[i, :] / HWt)
    
    BLHW[i] = (np.nansum(auxHW[1:]) / (len(auxHW) - 1)) * 100

    # Re-calculate quizzes according to the scheme
    auxq = []
    
    for q in Q[i, :]:
        
        if q == 100:
            
            auxq.append(100)
        
        elif q == 93.4:
            
            auxq.append(93.4)
            
        elif q == 88.12: 
            
            auxq.append(88.12)
        
        elif q == 86.8:
            
            auxq.append(86.8)
        
        elif q == 80.2:
            
            auxq.append(80.2)
        
        elif q == 73.6:
            
            auxq.append(73.6)
        
        elif q == 67:
            
            auxq.append(67)
        
        elif q == 0:
            
            auxq.append(0)

    auxq = np.sort(auxq)

    BLQ[i] = (np.nansum(auxq[2:]) / (len(auxq) - 2))

    # Calculate final
    BLF[i] = (np.sum(F[i])) / (np.sum(Ft)) * 100

    # Recalculate midterm if it is zero (didn't do it) or if it is smaller than final
    
    BLMT[i] = (np.sum(MT[i])) / (np.sum(MTt)) * 100
    
    if BLMT[i] == 0 or BLMT[i] < BLF[i]:
        
        BLMT[i] = BLF[i]

    # Calculate extra credit (participation)
    if EC3[i] >= 9:
        
        BEC[i] = 100 * 0.02
    
    elif 5 <= EC3[i] < 9:
        
        BEC[i] = 100 * 0.01
    
    else:
        
        BEC[i] = 0

    if EC2[i] == 1:
        
        BEC[i] += 100 * 0.01

    if EC1[i] == 1:
        
        BEC[i] += 100 * 0.01

# Grading: Homework 25%, Quizzes 5%, Midterm 30%, Final 40%, extra credit 0 to 4%

Final_grade_noc = (BLHW * 0.25 + BLQ * 0.05 + BLMT * 0.30 + BLF * 0.40 + BEC)

Final_grade_noc[Final_grade_noc > 100] = 100

# letter_noc = np.array(['']*len(Final_grade_noc))

letter_noc = np.full(len(Final_grade_noc), '', dtype=object)

for j in range(len(Final_grade_noc)):
    
    for i in range(len(letter)):
        
        if edges[i] <= Final_grade_noc[j] < edges[i+1]:
            
            letter_noc[j] = letter[i]
            
            break


# Exclude the max points, the students that didn't do anything (maybe they dropped the class) and the test student

exclude_indices = [0, 15, 104, 124]

# Create a logical index

logical_index = np.ones(len(Final_grade_noc), dtype=bool)

logical_index[exclude_indices] = False

# Evaluate all positions except the specified ones
evaluated_vector = Final_grade_noc[logical_index]

average_grade = np.mean(evaluated_vector)

print(f'Average grade is: {average_grade}')

# Final matrix
matrix = np.column_stack([BLQ, BLHW, BLMT, BLF, BEC, Final_grade_BL, letter_BL, Final_grade_noc, letter_noc])

final_matrix = np.column_stack([student_names, id_, SISUserID, SISLoginID, Section, matrix])

# Save the final matrix to an Excel file
final_df = pd.DataFrame(final_matrix)

final_df.to_csv('final_grades.csv', index=False, header=[
    'Student Name', 'ID', 'SIS User ID', 'SIS Login ID', 'Section',
    'Quiz', 'Assignment', 'Midterm', 'Exam', 'Participation',
    'Final Grade BL', 'Letter BL', 'Final Grade', 'Letter'
])


Average grade is: 89.32943798827468


In [114]:
grades_df[grades_df['Student'].str.contains('sanchez cruz', case=False, na=False)].iloc[:, 10:18]

,Quiz #1 (1594616),Quiz #2 (1594613),Quiz #8 (1614775),Quiz #3 (1594618),Quiz #4 (1594615),Quiz #5 (1594612),Quiz #6 (1594617),Quiz #7 (1594614)
92,80.20,93.40,73.6,80.20,72.72,86.80,80.20,NaN


In [115]:
(80.20+93.40+73.6+80.20+86.8+80.2)/6

82.4